# ASTR 310 Lecture 15 - astropy 2

### 1. Reading a table and converting coordinates/times (10 points total)

* Download the file `fermi_lat_grbs.dat` from today’s exercise page on the website. It contains 1000 gamma-ray bursts observed by the Fermi satellite between 2010 and 2018.

* Use the `astropy.io.ascii.read()` routine to read the file into a table. You need to specify the `header_start`, `data_start`, and `delimiter` arguments.

* Display the column headers using the `columns` or `colnames` method.  Print the units on the `ra` and `dec` columns.

[3 pts]

In [112]:
import astropy.io.ascii
import astropy.coordinates
import astropy.units as u
import numpy as np

In [113]:
table = astropy.io.ascii.read("fermi_lat_grbs.dat", guess=False, header_start=2, data_start=3, delimiter='|')
print(table.colnames)
print(table['ra'].unit)
print(table['dec'].unit)

['col0', 'name', 'ra', 'dec', 'trigger_time', 't90', 't90_error', 't90_start', 'fluence', 'fluence_error', 'flux_1024', 'flux_1024_error', 'flux_1024_time', 'flux_64', 'flux_64_error', 'flnc_band_ampl', 'flnc_band_ampl_pos_err', 'flnc_band_ampl_neg_err', 'flnc_band_epeak', 'flnc_band_epeak_pos_err', 'flnc_band_epeak_neg_err', 'flnc_band_alpha', 'flnc_band_alpha_pos_err', 'flnc_band_alpha_neg_err', 'flnc_band_beta', 'flnc_band_beta_pos_err', 'flnc_band_beta_neg_err', 'flnc_spectrum_start', 'flnc_spectrum_stop', 'pflx_best_fitting_model', 'pflx_best_model_redchisq', 'flnc_best_fitting_model', 'flnc_best_model_redchisq', '_1']
None
None


* Notice that RA and DEC are in sexagesimal format, represented as strings. Use `astropy.coordinates.Angle` to convert them to decimal degrees.  Print the table or try the `show_in_notebook` method.  (The method sometimes gets carried away, so you might like to restrict it to printing the first 10 rows.)  [3 pts]

In [114]:
table['ra'] = astropy.coordinates.Angle(table['ra'], unit=u.degree)
table['dec'] = astropy.coordinates.Angle(table['dec'], unit=u.degree)
table

col0,name,ra,dec,trigger_time,t90,t90_error,t90_start,fluence,fluence_error,flux_1024,flux_1024_error,flux_1024_time,flux_64,flux_64_error,flnc_band_ampl,flnc_band_ampl_pos_err,flnc_band_ampl_neg_err,flnc_band_epeak,flnc_band_epeak_pos_err,flnc_band_epeak_neg_err,flnc_band_alpha,flnc_band_alpha_pos_err,flnc_band_alpha_neg_err,flnc_band_beta,flnc_band_beta_pos_err,flnc_band_beta_neg_err,flnc_spectrum_start,flnc_spectrum_stop,pflx_best_fitting_model,pflx_best_model_redchisq,flnc_best_fitting_model,flnc_best_model_redchisq,_1
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,str12,float64,float64,str23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,float64,str9,float64,int64
--,GRB120830212,22.524666666666665,-80.03999999999999,2012-08-30 05:04:52.744,16.064,3.649,0.448,7.5151e-06,2.3629e-07,8.6556,0.4512,1.344,14.842,2.022,0.01907516,0.002260342,0.002260342,206.6716,19.34743,19.34743,-0.7385905,0.09011394,0.09011394,-2.908419,7.539405,7.539405,-1.024,17.408,pflx_comp,1.075,flnc_comp,0.891,--
--,GRB140323433,23.79727777777778,-79.905,2014-03-23 10:22:53.120,111.426,3.027,5.056,3.2424e-05,1.9208e-07,9.6741,0.2758,25.088,13.1489,1.2081,0.01208579,0.0005032783,0.0005032783,143.2164,21.11698,21.11698,-1.041237,0.02542675,0.02542675,-5.704211,23.21102,23.21102,-2.048,129.02,pflx_comp,0.955,flnc_comp,1.131,--
--,GRB181007737,14.081333333333333,-79.69,2018-10-07 17:41:28.372,23.552,0.362,-0.256,3.0597e-06,3.0382e-08,2.8347,0.2137,-0.192,4.4329,0.941,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
--,GRB100325246,13.942666666666668,-79.1,2010-03-25 05:54:43.949,8.192,1.086,-1.536,1.334e-06,1.7358e-08,4.9033,0.2367,-0.512,6.4777,0.9864,0.04793454,0.02380955,0.02380955,34.98019,5.283089,5.283089,-0.6888088,0.298304,0.298304,-2.467626,0.2204204,0.2204204,-2.048,7.168,pflx_comp,1.089,flnc_comp,1.003,--
--,GRB100507577,0.19333333333333333,-79.01,2010-05-07 13:51:15.728,44.033,5.221,-1.024,9.9681e-06,1.1543e-07,3.2911,0.2039,15.168,4.7504,0.95,0.04973427,0.007101182,0.007101182,133.6812,7.531464,7.531464,0.4626522,0.1081798,0.1081798,-3.466067,1.403857,1.403857,-13.312,31.744,pflx_plaw,1.157,flnc_comp,0.998,--
--,GRB171206122,0.632,-78.2,2017-12-06 02:55:44.937,15.36,3.083,-1.536,1.5221e-06,7.2998e-08,4.8222,0.4886,0.448,11.4606,3.5981,0.007276174,0.001115594,0.001115594,629.9004,209.734,209.734,-1.292658,0.1413209,0.1413209,-6.331875,379.0621,379.0621,-1.024,3.072,pflx_plaw,0.989,flnc_comp,1.001,--
--,GRB140518709,16.269333333333332,-77.86,2014-05-18 17:00:43.425,0.704,0.466,-0.64,3.8053e-07,2.6848e-08,2.454,0.3526,-1.024,5.3743,1.1495,5633.034,221743.6,221743.6,182.9023,69.33234,69.33234,26.63325,73.72793,73.72793,-1.975177,0.3770473,0.3770473,-0.192,-0.128,pflx_plaw,1.026,flnc_plaw,1.025,--
--,GRB130623488,1.381527777777778,-77.78388888888888,2013-06-23 11:42:47.046,22.272,0.923,-1.792,2.4645e-06,3.0042e-08,5.2588,0.2332,13.888,8.3458,1.2823,0.01909407,0.02033522,0.02033522,28.06861,4.528196,4.528196,-1.095303,0.3499717,0.3499717,-2.362571,0.1704245,0.1704245,-2.048,20.48,pflx_plaw,1.142,flnc_comp,1.061,--


* Do some brief analysis:  
  a) Construct a boolean mask array that selects only those bursts with durations $t_{90} < 2$ seconds and relative errors in the duration $\frac{t_{90\_error}}{t_{90}} < 50\%$.  
  b) Construct a second boolean mask array that selects $t_{90} > 2$ seconds with the same relative error.  
  c) Compare the median peak energy (flnc_band_epeak, in keV) of the two samples. Use the NumPy `median` function to compute the values. [4 pts]

In [115]:
table['t90'].unit = u.second
table['t90_error'].unit = u.second
table['flnc_band_epeak'].unit = u.keV

mask1 = (table['t90'] < 2*u.second) & ((table['t90_error'] / table['t90']) < 0.5)
mask2 = (table['t90'] > 2*u.second) & ((table['t90_error'] / table['t90']) < 0.5)

print(np.median(table['flnc_band_epeak'][mask1].quantity))
print(np.median(table['flnc_band_epeak'][mask2].quantity))

359.3643 keV
135.48020000000002 keV


### 2. Working with and modifying a table (10 points total)

Download the files "mcxc.dat" and "mcxc.readme" from today’s exercise page on the website. These files contain a catalog of X-ray-detected clusters of galaxies from Piffaretti et al. (2011) obtained through the VizieR service at the University of Strasbourg.

1. Use `astropy.io.ascii.read()` to read the table and its metadata into Python. This table is in "CDS" format, and you specify the metadata file using the `readme` argument.

2. Extract the log of L500 (luminosity in units of $10^{44} \text{erg/s}$), log of M500 (mass in $10^{14}$ solar masses), and z (redshift) columns from the data into 1D arrays. Create a mask array selecting those clusters with redshift $< 0.1$.

3. Now construct the array $r$ containing $r = \log L - 1.64 \log M$. The X-ray luminosity and mass of galaxy clusters are correlated roughly such that $L \propto M^{5/3}$, so the range in $r$ is small.  
  
4. Modify the table to add a masked column (`MaskedColumn` object) for $r$, and use the description "$r = \log L - 1.64 \log M$" for that column. Use the mask array you created to mask those clusters you don't want to store $r$ values for. Print the summary information for the table, and print the table to check whether your column was added correctly. You should see "--" for masked-out rows.  

5. Write the modified table to a file named "mcxc_new.csv" in comma-separated value (CSV) format.  




In [116]:
import astropy.table


t = astropy.io.ascii.read("mcxc.dat", readme="mcxc.readme", format='cds')
log_l500 = np.log10(t['L500'])
log_m500 = np.log10(t['M500'])
z = t['z']

z_mask = z < 0.1

r = log_l500 - 1.64*log_m500

t['r'] = astropy.table.MaskedColumn(r, mask=~z_mask)
t['r'].description = "r = log(L) - 1.64 * log(M)"

In [117]:
t

MCXC,OName,AName,RAh,RAm,RAs,DE-,DEd,DEm,DEs,RAdeg,DEdeg,GLON,GLAT,z,Cat,Sub-Cat,Scale,L500,M500,R500,Notes,Cat1,Cat2,Cat3,Cat4,L500r1,L500r2,L500r3,L500r4,r
,,,h,min,s,,deg,arcmin,arcsec,deg,deg,deg,deg,,,,kpc / arcsec,1e+37 W,1e+14 Msun,Mpc,,,,,,,,,,1e+37 W
str12,str18,str54,int64,int64,float64,str1,int64,int64,int64,float64,float64,float64,float64,float64,str12,str12,float64,float64,float64,float64,str42,str12,str12,str12,str12,float64,float64,float64,float64,float64
J0000.1+0816,RXC J0000.1+0816,UGC 12890,0,0,7.1,--,8,16,28,0.03,8.274,101.783,-52.477,0.0396,NORAS/REFLEX,NORAS,0.784,0.19628,0.7373,0.6296,--,BCS,--,--,--,1.084,--,--,--,-0.49006049429220355
J0000.4-0237,RXC J0000.4-0237,--,0,0,24.7,-,2,37,30,0.103,-2.625,94.268,-62.622,0.0379,SGP,SGP,0.752,0.052338,0.3297,0.4817,--,--,--,--,--,--,--,--,--,-0.4908979504552985
J0001.6-1540,RXC J0001.6-1540,--,0,1,39.0,-,15,40,52,0.412,-15.681,75.129,-73.733,0.1246,SGP,SGP,2.234,0.814902,1.6557,0.8021,--,--,--,--,--,--,--,--,--,--
J0001.9+1204,RXC J0001.9+1204,A2692,0,1,57.0,--,12,4,23,0.488,12.073,104.308,-49.001,0.2033,NORAS/REFLEX,NORAS,3.342,1.990205,2.6927,0.9178,--,--,--,--,--,--,--,--,--,--
J0003.1-0605,RXCJ0003.1-0605,A2697,0,3,11.8,-,6,5,10,0.799,-6.086,92.169,-66.033,0.232,NORAS/REFLEX,REFLEX,3.698,6.107094,5.2188,1.1326,--,SGP,--,--,--,0.952,--,--,--,--
J0003.2-3555,RXCJ0003.2-3555,A2717,0,3,12.1,-,35,55,38,0.801,-35.927,349.33,-76.49,0.049,NORAS/REFLEX,REFLEX,0.959,0.441862,1.2016,0.7388,losStr,SGP,--,--,--,0.886,--,--,--,-0.4855196142700461
J0003.8+0203,RXCJ0003.8+0203,A2700,0,3,50.6,--,2,3,48,0.961,2.063,99.61,-58.637,0.0924,NORAS/REFLEX,REFLEX,1.719,0.846881,1.7342,0.8233,--,eBCS,SGP,--,--,0.92,0.922,--,--,-0.46430026883136716
J0004.9+1142,RXC J0004.9+1142,UGC 00032,0,4,59.4,--,11,42,2,1.247,11.701,105.239,-49.569,0.0761,NORAS/REFLEX,NORAS,1.443,0.518795,1.3009,0.752,--,eBCS,--,--,--,0.967,--,--,--,-0.4723642366254167


In [118]:
t.write("mcxc_new.csv", format='csv', overwrite=True)